# PyTorch Workflow

The PyTorch end-to-end workflow

Resources:
* Github Repo - https://github.com/mrdbourke/pytorch-deep-learning
* Web Version - https://www.learnpytorch.io/01_pytorch_workflow/

In [4]:
what_were_covering = {1: "data (prepare and load)",
                    2: "build model",
                    3: "fitting the model to data (training)",
                    4: "making prediction and evaluating (inference)",
                    5: "saving and loading a model",
                    6: "putting it together"}

what_were_covering

{1: 'data (prepare and load)',
 2: 'build model',
 3: 'fitting the model to data (training)',
 4: 'making prediction and evaluating (inference)',
 5: 'saving and loading a model',
 6: 'putting it together'}

In [5]:
import torch
from torch import nn # nn contains all of PyTorch's building blocks for neural networks
import matplotlib.pyplot as plt

# Check PyTorch Version
print(torch.__version__)

1.13.1


## 1. Data (preparing and loading)

Data can be almost anything in machine learning

* Excel Spreadsheet
* Images of any kind
* Videos (youtube etc.)
* Audio (songs, podcasts)
* DNA
* Text

Machine learning is a game of two parts:
1. Get data into numerical representation.
2. Build a model to learn patterns in that numerical representation.

To showcase this, let's create some *known* data using the linear regression formula.

We'll use a linear regression formula to make a straight line with *known* **parameters**.

In [6]:
# Create known parameters
# f(x) = a + bx
weight = 0.7
bias = 0.3

# Create Inputs
start = 0
end = 1
step = 0.02

# add extra bracket (dim) to X
X = torch.arange(start, end, step).unsqueeze(dim=1) # uppercase = matrix || tensor

# output per input
y = weight * X + bias # lowercase = vector

X[:10], y[:10], len(X), len(y)

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]),
 50,
 50)

### Splitting data into training and test sets
1. Training Set (60~80%)
2. Validation Set (10~20%)
3. Testing Set (10~20%)


In [ ]:
# Create